In [ ]:
#Dataset from Kaggle: https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("diabetes.csv") # this path is generalized, please input original local file path here
df

In [ ]:
df.describe(include='all')
#Look at descriptive stats to understand data and identify any inconsistency, missing values, and outliers

In [ ]:
df.hist(figsize=(12, 10), bins=30, edgecolor='black')
#Look at histogram to understand spread of data

In [ ]:
df.boxplot(figsize=(12,5), vert=False)
#Look at boxplot to understand spread of data

In [ ]:
# Missing Values:
#1. BloodPressue, SkinThickness, Insulin, BMI, and Age have a Min of 0, which is impossible and indicate missing value

# Outliers:
#2. BloodPressure at 122 (diastollic) is possible but indicates medical emergency
#3. SkinThickness of 99mm is unrealistic
#4. Insulin at 846 is unrealistic
#5. BMI at 67 is rare

In [ ]:
#Assign 0 as missing values for Glucose, BloodPressue, SkinThickness, Insulin, BMI

columns_missing_value = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

df[columns_missing_value] = df[columns_missing_value].replace(0, np.nan)

df[columns_missing_value]

In [ ]:
df[columns_missing_value].boxplot(figsize=(12,5), vert=False)

In [ ]:
df[columns_missing_value].isnull().sum()

In [ ]:
df[columns_missing_value].isnull().sum()/ len(df)*100

In [ ]:
#SkinThickness and Insulin have a high percentage of missing values, replacing it with mean may affect modelling results later on, hence we should replace it with predicted values.
correlation_matrix = df.corr()
correlation_matrix

In [ ]:
#Identify correlation of SkinThickness and Insulin with other fields for prediction
correlation_matrix[["SkinThickness", "Insulin"]].abs() > 0.2

In [ ]:
#Select features use to predict missing values
correlated_features = correlation_matrix[["SkinThickness", "Insulin"]].abs() > 0.2
selected_features = correlation_matrix.index[correlated_features.any(axis=1)].tolist()
selected_features.remove("SkinThickness")
selected_features.remove("Insulin")
print(f"Selected features for imputation: {selected_features}")

In [ ]:
#Define function
def impute_missing_values(df, target_column, feature_columns):
    #Split the data into missing and non-missing, so we can predict missing values using non-missing data
    df_train = df[df[target_column].notna()] 
    df_missing = df[df[target_column].isna()]  
    
    if not df_missing.empty:
        # Define X (predictors) and y (target)
        X_train = df_train[feature_columns]
        y_train = df_train[target_column]
        
        # Train regression model
        model = RandomForestRegressor()
        model.fit(X_train, y_train)

        # Predict missing values
        X_missing = df_missing[feature_columns]
        df.loc[df[target_column].isna(), target_column] = model.predict(X_missing)

In [ ]:
# Impute missing values for SkinThickness and Insulin
impute_missing_values(df, "SkinThickness", selected_features)
impute_missing_values(df, "Insulin", selected_features)

In [ ]:
print(df.isnull().sum())

In [ ]:
#Replace remaining missing values (Glucose, BloodPressure and BMI) with mean
df.fillna(df.mean(), inplace=True)
df[columns_missing_value].describe()

In [ ]:
print(df.isnull().sum())

In [ ]:
#Replace outliers for SkinThickness, Insulin, BloodPressue, and BMI with outlier cutoff value at IQR Upper Bound:
def replace_outliers_with_iqr(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR
    df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])
    return df

In [ ]:
outlier_columns = ["SkinThickness", "Insulin", "BloodPressure", "BMI"]

In [ ]:
for column in outlier_columns:
    df = replace_outliers_with_iqr(df, column)

In [ ]:
df.boxplot(figsize=(12,5), vert=False)

In [ ]:
df.describe()

In [ ]:
#Save the cleaned dataset
df.to_csv("cleaned_diabetes.csv", index=False)

In [ ]:
#Exploration: Finding out number of diabetic patients by obesity level
#First, split bmi into categories
def bmi_categories(bmi):
    if bmi < 18.5:
        return "Underweight"
    elif 18.5 <= bmi < 25:
        return "Normal"
    elif 25 <= bmi < 30:
        return "Overweight"
    else:
        return "Obese"

In [ ]:
df['Obesity_Level'] = df['BMI'].apply(bmi_categories)
df

In [ ]:
obesity_diabetes_count = df[df["Outcome"] == 1]["Obesity_Level"].value_counts()
obesity_diabetes_count

In [ ]:
plt.figure(figsize=(10,5))
colors = ["orange", "blue", "green"]
ax = sns.barplot(x=obesity_diabetes_count.index, y=obesity_diabetes_count.values, palette=colors)
for container in ax.containers:
    ax.bar_label(container)
plt.xlabel("Obesity Level")
plt.ylabel("Number of Diabetic Patients")
plt.title("Number of Diabetic Patients by Obesity Level",fontsize=12, fontweight="bold")
plt.show()